# Flip-Chip Devices in Qiskit Metal

In this notebook, we will construct a device made of a coupled qubit - resonator system in a flip-chip environment. After that, we proceed to analyse them using the EPR technique available in the qiskit metal analyses folder.

The flip-chip device in this notebook is very basic, consisting of two chips separated by a specific distance (that we shall specify later). There are different layout flavours, one way is to place the resonator at the 'bottom' chip or so-called 'control' chip, and the qubit on the 'top' chip or so-called 'qubit' chip.


## 1. Building a Flip-Chip Device Design
We use another class called the <code>DesignFlipChip</code> class. The design workflow is very similar to the one for building planar devices (cf. <code>DesignPlanar</code>), except that there are two chips, and you need to specify which chip you want to draw the device on. At the moment, the two chips are called the 'C_chip' and the 'Q_chip'.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
#from qiskit_metal.qlibrary.resonator.readoutres_fc import ReadoutResFC

# suppress deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=Warning)

In [3]:
# Initialise design
design = designs.DesignFlipChip()
# Specify design name
design.metadata['design_name'] = 'FlipChip_Device'
# launch GUI
gui = MetalGUI(design)
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

Next, we draw an Xmon on the Q-chip, and a readout resonator on the C-chip.

In [4]:
# delete existing components
design.delete_all_components()

# draw an Xmon on the Q_chip. Notice that I have defined one more item called chip and set it to be the 'Q_chip'
options_cpads = Dict(
        connector_type = '0', # claw type
        claw_length = '30um',
        ground_spacing = '5um',
        claw_width = '10um',
        claw_gap = '6um')
options = Dict(
    chip = 'Q_chip',
    cross_width = '20um',
    cross_length = '150um',
    cross_gap = '20um',
    connection_pads = Dict(
        claw_west = Dict(connector_location= '180', **options_cpads)))

# position the center of the Xmon at (0,0)
q1_x = '0.0 mm'
q1_y = '0.0 mm'

# build the device, positioned at (q1_x, q1_y)
q1 = TransmonCross(design, 'Q1', options = Dict(pos_x=q1_x, pos_y=q1_y, **options))

# rebuild in gui
gui.rebuild()
gui.autoscale()

In [5]:
# draw a readout resonator on the C_chip. Note that we specify the chip='C_chip'.
# The readout resonator can me made into any shape. Here I opted to use a lambda/4 cpw-based resonator.
# The open-ended end of the resonator has a circular shape with a radius of readout_radius. Change this radius to change the qubit-resonator coupling strength.
# The cpw_width and cpw_gap are the size of the signal line and the ground gap of the cpw transmission line.
# The parameters readout_l1 ... l5, are the lengths of various parts of the cpw.
# arc_step is the length of a line that approximates an arc. 
options = Dict(
    chip = 'C_chip',
    readout_radius = '20 um',
    readout_cpw_width = '10 um',
    readout_cpw_gap = '10 um',
    readout_cpw_turnradius = '50 um',
    readout_l1 = '600 um',
    readout_l2 = '200 um',
    readout_l3 = '300 um',
    readout_l4 = '150 um',
    readout_l5 = '550 um',
    arc_step = '5 um')

# the resonator is set to have its origin at the center of the circular patch.
# So we set the qubit and the resonator to share the same coordinate (q1_x, q1_y)
r1 = ReadoutResFC(design, 'R1', options = Dict(pos_x = q1_x, pos_y = q1_y, **options))

# rebuild in gui
gui.rebuild()
gui.autoscale()


NameError: name 'ReadoutResFC' is not defined

## 2. Preparation Prior to Rendering in ANSYS


As described before, we have specified two chips in the <code>FlipChipDesign</code> class, i.e. C_chip and Q_chip. This information is stored in the chips variables. 

In [6]:
design.chips

{'C_chip': {'material': 'silicon',
  'layer_start': '0',
  'layer_end': '2048',
  'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'center_z': '0.0mm',
   'size_x': '9mm',
   'size_y': '9mm',
   'size_z': '-280um',
   'sample_holder_top': '890um',
   'sample_holder_bottom': '1650um'}},
 'Q_chip': {'material': 'silicon',
  'layer_start': '0',
  'layer_end': '2048',
  'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'center_z': '20 um',
   'size_x': '9mm',
   'size_y': '9mm',
   'size_z': '280um',
   'sample_holder_top': '890um',
   'sample_holder_bottom': '1650um'}}}

It is important to note that in terms of component rendering in ANSYS, the components are drawn on an XY plane with its Z-coordinate being defined by 'center_z'. For the design object we just created, C_chip components are drawn at z=0, and the Q_chip components are drawn at 20 um.

In [7]:
design.chips['C_chip']['size']['center_z']

'0.0mm'

In [8]:
design.chips['Q_chip']['size']['center_z']

'20 um'

Next, we set the Q_chip to be at a distance of, say 10 um, from the C_chip.

In [9]:
design.chips['Q_chip']['size']['center_z'] = '10 um'
design.chips['Q_chip']['size']['center_z']

'10 um'

It is possible that you would like to assign a new material to the chip substrate. In this case, I'd usually define a new material in the library instead of modifying the existing one. If this is not of interest, feel free to skip this part.

Say, we would like to have a Silicon substrate with a different dielectric constant (11.45 instead of the default value of 11.9). Let's call this new material 'Si_11.45'.

To do this, first we add the new material to the ANSYS material library (either manually or scripted like what we did below), and then specify the new material in the design object.

In [13]:
# new material properties
newMaterial_name = 'Si_11.45'
newMaterial_permittivity = '11.45'
newMaterial_losstangent = '1e-7'

In [14]:
# add the new material via scripting
# uncomment if needed
from pyEPR import ansys, project_info
# get the handles to the ANSYS system
ans_prj = project_info.ProjectInfo()
oProject = ans_prj.project._project
defmanager = oProject.GetDefinitionManager()
# check if the material exists
if not defmanager.DoesMaterialExist(newMaterial_name):
    defmanager.AddMaterial(
        [
            "NAME:%s"%(newMaterial_name),
            "CoordinateSystemType:=", "Cartesian",
            "BulkOrSurfaceType:=", 1,
            [
                    "NAME:PhysicsTypes",
                    "set:=", ["Electromagnetic"]
            ],
            "permittivity:=", "%s"%(newMaterial_permittivity),
            "dielectric_loss_tangent:=", "%s"%(newMaterial_losstangent)
        ])
# release the handle to ANSYS
ans_prj.project.release()
ans_prj.desktop.release()
ans_prj.app.release()
ansys.release()

INFO 04:12PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:12PM [load_ansys_project]: 	Opened Ansys App
INFO 04:12PM [load_ansys_project]: 	Opened Ansys Desktop v2023.1.0
INFO 04:12PM [load_ansys_project]: 	Ansys Project was not found.
	 Project is None.
INFO 04:12PM [connect]: 	Connection to Ansys NOT established.  

INFO 04:12PM [connect]: 	 Project not detected in Ansys. Is there a project in your desktop app? 🤔 

INFO 04:12PM [connect]: 	 Connected to project "None". No design detected


AttributeError: 'NoneType' object has no attribute '_project'

In [ ]:
# Specify the new material assigment for the chip
design.chips['C_chip']['material'] = newMaterial_name
design.chips['Q_chip']['material'] = newMaterial_name

## 3.  Run EPR Analysis
We proceed to run the EPR analysis. This follows closely tutorial 4.02 Eigenmode and EPR.

First, we initialise an EPRanalysis class object, and update the ANSYS simulation setup.

In [15]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_fc = EPRanalysis(design, "hfss")
eig_fc.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'min_freq_ghz': 1,
 'n_modes': 1,
 'max_delta_f': 0.5,
 'max_passes': 10,
 'min_passes': 1,
 'min_converged': 1,
 'pct_refinement': 30,
 'basis_order': 1,
 'vars': {'Lj': '10 nH', 'Cj': '0 fF'}}

In [16]:
# update setup setting
eig_fc.sim.setup.max_passes = 20
eig_fc.sim.setup.vars.Lj = '10 nH'
# since we are analysing both the qubit and the resonator, we set the number of modes to be 2
eig_fc.sim.setup.n_modes = 2
# if there is not enough computational power, you can set a less stringent max_delta_f
eig_fc.sim.setup.max_delta_f = 1 
eig_fc.sim.setup.min_freq_ghz = 1.5
# pass the name of the junction (for the EPR analysis purpose)
eig_fc.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'
eig_fc.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'

eig_fc.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'min_freq_ghz': 1.5,
 'n_modes': 2,
 'max_delta_f': 1,
 'max_passes': 20,
 'min_passes': 1,
 'min_converged': 1,
 'pct_refinement': 30,
 'basis_order': 1,
 'vars': {'Lj': '10 nH', 'Cj': '0 fF'}}

Next, we run the ANSYS simulation and then the EPR analysis.

In [17]:
eig_fc.sim.run(name="qubitres", components=['Q1', 'R1'], open_terminations=[], box_plus_buffer = True)

INFO 04:12PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:12PM [load_ansys_project]: 	Opened Ansys App
INFO 04:12PM [load_ansys_project]: 	Opened Ansys Desktop v2023.1.0
INFO 04:12PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/askev/OneDrive/文件/Ansoft/
	Project:   Project7
INFO 04:12PM [connect_design]: No active design found (or error getting active design).
INFO 04:12PM [connect]: 	 Connected to project "Project7". No design detected
INFO 04:12PM [connect_design]: 	Opened active design
	Design:    qubitres_hfss [Solution type: Eigenmode]
WARNING 04:12PM [connect_setup]: 	No design setup detected.
WARNING 04:12PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:12PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
04:12PM 39s WARNING [get_unique_component_ids]: The component=R1 in highlight_qcomponents not in QDesign.
04:12PM 39s WARNING [render_design]: Unable to proceed with rendering. Please check selection.
IN

com_error: (-2147352567, '發生例外狀況。', (0, None, None, None, 0, -2147024349), None)

In [ ]:
eig_fc.setup.dissipatives.dielectrics_bulk = ['Q_chip', 'C_chip']
eig_fc.run_epr()

In [ ]:
eig_fc.sim.close()